In [ ]:
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pymongo
import time
import pandas as pd



conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


In [ ]:
# Define database and collection
db = client.mars_db
collection = db.pages

In [ ]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [ ]:
articles = soup.find_all('div', class_='slide')

for article in articles:
    title = article.find('div', class_='content_title').text
    paragraph = article.find('div', class_= 'rollover_description_inner').text
    
    print('---------------')
    print(title)
    print(paragraph)
    
    
    post = {
        'title': title,
        'paragraph': paragraph,
        }

    # Insert dictionary into MongoDB as a document
    collection.insert_one(post)


In [48]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.mars_db
collection2 = db.image


executable_path = {'executable_path': '/usr/local/bin/chromedriver'}

browser = Browser('chrome', **executable_path, headless=False)

url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)

browser.click_link_by_id('full_image')

time.sleep(1)

html = browser.html
soup = BeautifulSoup(html, "html.parser")

image_link = soup.find('img',class_= 'fancybox-image')['src']


featured_image_url =('https://www.jpl.nasa.gov/' + image_link) 
print('-----------')
print('https://www.jpl.nasa.gov/' + image_link)
    
post2 = {
    'image': featured_image_url
}
    
collection2.insert_one(post2)
    
    
browser.quit()    
    

-----------
https://www.jpl.nasa.gov//spaceimages/images/mediumsize/PIA16153_ip.jpg


In [35]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.mars_db
collection3 = db.facts

df = pd.read_html(str('http://space-facts.com/mars/'))[0]

df.columns = ['Description', 'Mars']

df.set_index('Description', inplace=True)

table_dict = df.to_dict()

collection3.insert_one(table_dict)


In [63]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}

browser = Browser('chrome', **executable_path, headless=False)

url4 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url4)

mars_hem = []

for x in range(1, 5):  
    
    browser.click_link_by_partial_text('Hemisphere')
    
    time.sleep(3)
    
    html = browser.html
    
    soup = BeautifulSoup(html, 'html.parser')
    
    images = soup.find_all('div', class_='downloads')
    
    browser.back()
    
    for image in images:
        ul = image.find('ul')
        li = ul.find('li')
        link = li.find('a')
        img_url = link['href']
        
        print('------------')
        print(img_url)
        
    

print('Scraping Complete')
    



------------
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
------------
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
------------
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
------------
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Scraping Complete
